In [1]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [3]:
algo = SVD()
algo.fit(trainset)

In [4]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.4839407060574272, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8725641394716637, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.098621279402123, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.650666842132194, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.637671795011912, details={'was_impossible': False})]

In [5]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.4839407060574272),
 ('882', '291', 3.8725641394716637),
 ('535', '507', 4.098621279402123)]

In [6]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.96   {'was_impossible': False}


In [7]:
accuracy.rmse(predictions)

RMSE: 0.9463


0.9462857777118172

## OS 파일 데이터를 Surprise 데이터 세트로 로딩

In [8]:
import pandas as pd 

ratings = pd.read_csv('./data/ratings.csv')

ratings.to_csv('./data/ratings_noh.csv', index=False, header=False)

In [9]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('./data/ratings_noh.csv', reader=reader)

In [10]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## Pandas DataFrame에서 Surprise 데이터 세트로 로딩

In [11]:
import pandas as pd 
from surprise import Reader, Dataset

ratings = pd.read_csv('./data/ratings.csv')
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## 교차 검증과 하이퍼 파라미터 튜닝

In [12]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./data/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8668  0.8745  0.8756  0.8732  0.8702  0.8721  0.0032  
MAE (testset)     0.6673  0.6710  0.6736  0.6716  0.6685  0.6704  0.0022  
Fit time          2.36    2.35    2.24    2.36    2.23    2.31    0.06    
Test time         0.05    0.05    0.09    0.11    0.05    0.07    0.03    


{'test_rmse': array([0.86676102, 0.87453259, 0.87561688, 0.87321953, 0.8702372 ]),
 'test_mae': array([0.66733703, 0.67098965, 0.673554  , 0.67164238, 0.66853808]),
 'fit_time': (2.3579678535461426,
  2.345672845840454,
  2.2409770488739014,
  2.3640899658203125,
  2.231494903564453),
 'test_time': (0.04751300811767578,
  0.04834890365600586,
  0.09394478797912598,
  0.11060309410095215,
  0.04688882827758789)}

In [13]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8773739382218069
{'n_epochs': 20, 'n_factors': 50}


## Surprise를 이용한 개인화 영화 추천 시스템 구출

In [14]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

data_folds = DatasetAutoFolds(ratings_file='./data/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [15]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [16]:
movies = pd.read_csv('./data/movies.csv')

movieIds = ratings[ratings['userId'] == 9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']== 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [17]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [18]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    total_movies = movies['movieId'].tolist()

    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수 :', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 : 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [19]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    top_movie_ids = [ int(pred.iid) for pred in top_predictions ]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수 : 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122758
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
